In [64]:
#import libs

import pandas as pd 


In [65]:
data = pd.read_csv("spamdata.csv", encoding='latin-1')
data.head()

,ï»¿label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [66]:
import string
punctuation = string.punctuation
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
from nltk.stem.wordnet import WordNetLemmatizer
lem =  WordNetLemmatizer()
def clean_data(input_data):
    #convert to lower case
    clean_text = input_data.lower()
    #remove punctuations
    clean_text = "".join(data for data in clean_text if data not in punctuation)
    #remove stop words.
    words = clean_text.split()
    words = [w for w in words if w not in stopwords_list]
    clean_text = " ".join(words)
    #use lametasation for getting the root word.
    #Generally we lametize nouns and verbs.
    words = [lem.lemmatize(word, "v") for word in words]
    words = [lem.lemmatize(word, "n") for word in words]
    clean_text = " ".join(words)
    return clean_text

clean_data("I will be dancing in rain. Baba black sheep do you have any wool ?")

'dance rain baba black sheep wool'

In [67]:
data['cleaned'] = data['text'].apply(clean_data)
data.head()

,ï»¿label,text,cleaned
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though


In [68]:
# feature engineering 

#meta features

data['word_count'] = data['text'].apply(lambda x : len(x.split()))
data['word_count_clean'] = data['cleaned'].apply(lambda x : len(x.split()))

data['char_count'] = data['text'].apply(lambda x: len(x.replace(" ","")))
data['char_count_clean'] = data['cleaned'].apply(lambda x: len(x.replace(" ","")))

data['num_dig']= data['text'].apply(lambda x : sum(1 if w.isdigit() else 0 for w in x.split()))


In [69]:
data.head()

,ï»¿label,text,cleaned,word_count,word_count_clean,char_count,char_count_clean,num_dig
0,ham,"Go until jurong point, crazy.. Available only ...",go jurong point crazy available bugis n great ...,20,16,92,67,0
1,ham,Ok lar... Joking wif u oni...,ok lar joke wif u oni,6,6,24,16,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,28,23,128,113,2
3,ham,U dun say so early hor... U c already then say...,u dun say early hor u c already say,11,9,39,27,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah dont think go usf live around though,13,8,49,33,0


In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cvz = CountVectorizer()
cvz.fit(data["cleaned"].values)
count_vectors = cvz.transform(data["cleaned"].values)


tfidfvz = TfidfVectorizer(max_features=500)
tfidfvz.fit(data["cleaned"].values)
tfidf_vectors = tfidfvz.transform(data["cleaned"].values)

In [71]:
count_vectors


<5572x8206 sparse matrix of type '<class 'numpy.int64'>'
	with 46827 stored elements in Compressed Sparse Row format>

In [72]:
tfidf_dic = dict(zip(tfidfvz.get_feature_names(),tfidfvz.idf_))
df_tfidf = pd.DataFrame(columns=["words", "word_tfidf_scores"]).from_dict(tfidf_dic, orient = "index")

In [73]:
df_tfidf.columns = [ "word_tfidf_scores"]
df_tfidf

,word_tfidf_scores
10,6.581166
100,5.936809
1000,5.754488
150,5.888019
150p,6.406813
...,...
yet,5.713666
yo,6.099328
youre,5.655397
yr,6.581166


In [74]:
from scipy.sparse import hstack, csr_matrix
metafeatures = ['word_count', 'word_count_clean','char_count', 'char_count_clean', 'num_dig']
feature_set1 =  data[metafeatures]

train = hstack([tfidf_vectors, csr_matrix(feature_set1)],'csr')
train

<5572x505 sparse matrix of type '<class 'numpy.float64'>'
	with 51677 stored elements in Compressed Sparse Row format>

In [75]:
from sklearn.preprocessing import LabelEncoder

labelencode=LabelEncoder()
target = data['ï»¿label']
target = labelencode.fit_transform(target)
target

array([0, 0, 1, ..., 0, 0, 0])

In [76]:
from sklearn.model_selection import train_test_split
train_text, test_text, train_label, test_label = train_test_split(train,target)

In [77]:
train_label.shape

(4179,)

In [78]:
#import ML: models

from sklearn import naive_bayes
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import ensemble
from sklearn.metrics import accuracy_score

In [79]:
def get_accuracy_of_model(model,train_x,train_y,val_x,val_y):
    model.fit(train_x,train_y)
    pred = model.predict(val_x)
    score=accuracy_score(pred,val_y)
    print("Name : {} Score :{}".format(model,score))
    


In [80]:
model = naive_bayes.MultinomialNB()
get_accuracy_of_model(model,train_text,train_label,test_text,test_label)


model = LogisticRegression()
get_accuracy_of_model(model,train_text,train_label,test_text,test_label)

model = svm.SVC()
get_accuracy_of_model(model,train_text,train_label,test_text,test_label)

Name : MultinomialNB() Score :0.9727207465900933
Name : LogisticRegression() Score :0.9763101220387652


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Name : SVC() Score :0.9396984924623115


In [81]:
model = ensemble.BaggingClassifier()
get_accuracy_of_model(model,train_text,train_label,test_text,test_label)


Name : BaggingClassifier() Score :0.9770279971284996


In [82]:
model= ensemble.ExtraTreesClassifier()
get_accuracy_of_model(model,train_text,train_label,test_text,test_label)

Name : ExtraTreesClassifier() Score :0.9834888729361091


In [83]:
model = ensemble.GradientBoostingClassifier()
get_accuracy_of_model(model,train_text,train_label,test_text,test_label)

Name : GradientBoostingClassifier() Score :0.9727207465900933
